In [4]:
import sys
assert sys.version_info >= (3, 5)

import sklearn
assert sklearn.__version__ >= "0.20"

import numpy as np
import pandas as pd
import os

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

np.random.seed(42)

In [15]:
trade = pd.read_csv('trade_add.csv')
stocks = pd.read_csv('stocks_add.csv')

In [16]:
trade=trade.drop("Unnamed: 0", axis=1)
stocks=stocks.drop("Unnamed: 0", axis=1)

In [9]:
stocks.columns

Index(['기준일자', '종목번호', '종목명', '20년7월TOP3대상여부', '시장구분', '표준산업구분코드_소분류', '거래량',
       '주식가격_만원단위', '종목고가 - 종목저가/종목시가', '주식발행수', '시가총액', 'year', 'month',
       'day', '기준년월'],
      dtype='object')

In [10]:
trade.columns

Index(['기준년월', '그룹번호', '그룹내고객수', '종목번호', '그룹내_매수여부', '그룹내_매도여부', '매도고객수',
       '평균매수수량', '평균매도수량', '매수가격_중앙값', '매도가격_중앙값',
       '매도가격_중앙값 - 매수가격_중앙값/매수가격_중앙값', '매수수량 - 매도수량', '배당률', '거래량',
       '주식가격_만원단위', '종목고가 - 종목저가/종목시가', '시가총액', '매수고객수'],
      dtype='object')

In [17]:
trade=trade[["기준년월",'그룹번호',"매수고객수","종목번호","매수고객수"]]
stocks=stocks[['기준일자', '종목번호','시장구분','표준산업구분코드_소분류', '거래량',
       '종목고가 - 종목저가/종목시가', '주식발행수', '시가총액', 'year', 'month','day']]

In [28]:
trade['month']=trade['기준년월']%100

In [29]:
trade.head()

,기준년월,그룹번호,매수고객수,종목번호,매수고객수,day,month
0,201907,MAD01,7,A000660,7,7,7
1,201907,MAD04,23,A000660,23,7,7
2,201907,MAD05,3,A000660,3,7,7
3,201907,MAD06,27,A000660,27,7,7
4,201907,MAD09,7,A000660,7,7,7


In [22]:
stocks.head()

,기준일자,종목번호,시장구분,표준산업구분코드_소분류,거래량,종목고가 - 종목저가/종목시가,주식발행수,시가총액,year,month,day
0,20190701,A000020,1,의약품 제조업,31341,1.652893,27931470.0,2.725415e+07,2019,7,201
1,20190701,A000080,1,알코올음료 제조업,127186,1.703163,70133611.0,1.425628e+08,2019,7,201
2,20190701,A000100,1,의약품 제조업,395757,6.756757,66856810.0,1.712882e+09,2019,7,201
3,20190701,A000120,1,도로 화물 운송업,62513,2.930403,22812344.0,3.087602e+08,2019,7,201
4,20190701,A000140,1,기타 금융업,7661,3.090508,23206765.0,2.088942e+07,2019,7,201


In [48]:
import category_encoders as ce

#Create object for binary encoding
encoder= ce.BinaryEncoder(cols=['표준산업구분코드_소분류'], return_df=True)

#Fit and Transform Data 
data_encoded=encoder.fit_transform(stocks) 
stocks=data_encoded

In [49]:
MAD01=trade[trade['그룹번호']=='MAD01']

In [50]:
MAD01_merge=MAD01.merge(stocks, on=['month','종목번호'], how='right')
MAD01_merge

,기준년월,그룹번호,매수고객수,종목번호,매수고객수,day_x,month,기준일자,시장구분,표준산업구분코드_소분류_0,...,표준산업구분코드_소분류_4,표준산업구분코드_소분류_5,표준산업구분코드_소분류_6,표준산업구분코드_소분류_7,거래량,종목고가 - 종목저가/종목시가,주식발행수,시가총액,year,day_y
0,201907.0,MAD01,7.0,A000660,7.0,7.0,7,20190701,1,0,...,1,0,0,1,3538183,1.830986,728002365.0,5.118235e+09,2019,201
1,201907.0,MAD01,7.0,A000660,7.0,7.0,7,20190702,1,0,...,1,0,0,1,2495399,3.730273,728002365.0,5.161186e+09,2019,201
2,201907.0,MAD01,7.0,A000660,7.0,7.0,7,20190703,1,0,...,1,0,0,1,2790684,3.797468,728002365.0,5.084828e+09,2019,201
3,201907.0,MAD01,7.0,A000660,7.0,7.0,7,20190704,1,0,...,1,0,0,1,1503707,2.011494,728002365.0,5.096293e+09,2019,201
4,201907.0,MAD01,7.0,A000660,7.0,7.0,7,20190705,1,0,...,1,0,0,1,2043271,3.698435,728002365.0,5.043109e+09,2019,201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286056,NaN,NaN,NaN,A353810,NaN,NaN,7,20200722,0,0,...,1,0,1,0,139723,8.506616,2286728.0,1.243553e+07,2020,202
286057,NaN,NaN,NaN,A353810,NaN,NaN,7,20200723,0,0,...,1,0,1,0,110583,7.441016,2286728.0,1.264898e+07,2020,202
286058,NaN,NaN,NaN,A353810,NaN,NaN,7,20200724,0,0,...,1,0,1,0,97763,4.247788,2286728.0,1.292460e+07,2020,202
286059,NaN,NaN,NaN,A353810,NaN,NaN,7,20200727,0,0,...,1,0,1,0,358288,17.277487,2286728.0,1.385830e+07,2020,202


In [96]:
df=MAD01_merge.dropna(axis=0, thresh=20)

In [97]:
df=df[df['종목번호']=="A000660"]

In [98]:
df=df.reset_index()
df["datetime"] = pd.to_datetime(df["기준일자"], format='%Y%m%d')
df=df[["datetime", '매수고객수', 
       '시장구분', '표준산업구분코드_소분류_0', '표준산업구분코드_소분류_1', '표준산업구분코드_소분류_2',
       '표준산업구분코드_소분류_3', '표준산업구분코드_소분류_4', '표준산업구분코드_소분류_5', '표준산업구분코드_소분류_6',
       '표준산업구분코드_소분류_7', '거래량', '종목고가 - 종목저가/종목시가', '주식발행수', '시가총액','매수고객수']]

In [99]:
MAD01_test=df[df['datetime'].dt.month==6]
MAD01_test

,datetime,매수고객수,매수고객수,시장구분,표준산업구분코드_소분류_0,표준산업구분코드_소분류_1,표준산업구분코드_소분류_2,표준산업구분코드_소분류_3,표준산업구분코드_소분류_4,표준산업구분코드_소분류_5,표준산업구분코드_소분류_6,표준산업구분코드_소분류_7,거래량,종목고가 - 종목저가/종목시가,주식발행수,시가총액,매수고객수,매수고객수
225,2020-06-01,62.0,62.0,1,0,0,0,0,1,0,0,1,4205357,1.948843,728002365.0,6.053945e+09,62.0,62.0
226,2020-06-02,62.0,62.0,1,0,0,0,0,1,0,0,1,3114162,1.088271,728002365.0,6.049819e+09,62.0,62.0
227,2020-06-03,62.0,62.0,1,0,0,0,0,1,0,0,1,11642239,6.954436,728002365.0,6.328314e+09,62.0,62.0
228,2020-06-04,62.0,62.0,1,0,0,0,0,1,0,0,1,6678244,5.000000,728002365.0,6.488782e+09,62.0,62.0
229,2020-06-05,62.0,62.0,1,0,0,0,0,1,0,0,1,4000557,3.502825,728002365.0,6.528181e+09,62.0,62.0
230,2020-06-08,62.0,62.0,1,0,0,0,0,1,0,0,1,3789382,2.580645,728002365.0,6.656417e+09,62.0,62.0
231,2020-06-09,62.0,62.0,1,0,0,0,0,1,0,0,1,3410081,3.186813,728002365.0,6.569446e+09,62.0,62.0
232,2020-06-10,62.0,62.0,1,0,0,0,0,1,0,0,1,3045250,1.893096,728002365.0,6.574383e+09,62.0,62.0
233,2020-06-11,62.0,62.0,1,0,0,0,0,1,0,0,1,5467344,3.135498,728002365.0,6.466391e+09,62.0,62.0
234,2020-06-12,62.0,62.0,1,0,0,0,0,1,0,0,1,4742329,1.538462,728002365.0,6.200086e+09,62.0,62.0


In [100]:
MAD01_train=df[(df['datetime'].dt.month<6)&(df['datetime'].dt.month>=2)]

In [101]:
print(MAD01_test.shape, MAD01_train.shape)

(22, 18) (81, 18)


In [102]:
MAD01_train=MAD01_train.set_index('datetime')
MAD01_test=MAD01_test.set_index('datetime')

Data Scaling

In [103]:
y_train=MAD01_train['매수고객수'].values
X_train=MAD01_train.drop('매수고객수',axis=1).values

In [108]:
y_test=MAD01_test['매수고객수'].values
X_test=MAD01_test.drop('매수고객수',axis=1).values

In [109]:
y_train.shape

(81, 4)

In [110]:
X_train.shape

(81, 13)

In [112]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
scaler.fit(X_test)
X_test_scaled = scaler.transform(X_test)

In [113]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators=100, random_state=0)

In [115]:
X_train, X_test = X_train_scaled, X_test_scaled

regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)
print(y_pred)
#     print("테스트 세트 f1score:{:.2f}".format(f1score(y_pred, y_true)))

[[50.52 50.52 50.52 50.52]
 [50.72 50.72 50.72 50.72]
 [39.92 39.92 39.92 39.92]
 [34.17 34.17 34.17 34.17]
 [33.72 33.72 33.72 33.72]
 [33.27 33.27 33.27 33.27]
 [33.63 33.63 33.63 33.63]
 [33.   33.   33.   33.  ]
 [35.61 35.61 35.61 35.61]
 [50.24 50.24 50.24 50.24]
 [44.4  44.4  44.4  44.4 ]
 [50.76 50.76 50.76 50.76]
 [42.68 42.68 42.68 42.68]
 [46.51 46.51 46.51 46.51]
 [50.84 50.84 50.84 50.84]
 [50.14 50.14 50.14 50.14]
 [51.   51.   51.   51.  ]
 [41.38 41.38 41.38 41.38]
 [50.5  50.5  50.5  50.5 ]
 [50.86 50.86 50.86 50.86]
 [51.62 51.62 51.62 51.62]
 [45.84 45.84 45.84 45.84]]


In [116]:
y_test

array([[62., 62., 62., 62.],
       [62., 62., 62., 62.],
       [62., 62., 62., 62.],
       [62., 62., 62., 62.],
       [62., 62., 62., 62.],
       [62., 62., 62., 62.],
       [62., 62., 62., 62.],
       [62., 62., 62., 62.],
       [62., 62., 62., 62.],
       [62., 62., 62., 62.],
       [62., 62., 62., 62.],
       [62., 62., 62., 62.],
       [62., 62., 62., 62.],
       [62., 62., 62., 62.],
       [62., 62., 62., 62.],
       [62., 62., 62., 62.],
       [62., 62., 62., 62.],
       [62., 62., 62., 62.],
       [62., 62., 62., 62.],
       [62., 62., 62., 62.],
       [62., 62., 62., 62.],
       [62., 62., 62., 62.]])